<a href="https://colab.research.google.com/github/tomersk/learn-python/blob/main/08_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8. Input-Output
## 8.1 xls
The data.xls file contains the data of soil moisture estimated from the AMSR-E platform. You can open the xls file and have a look at its content. In this file we have data in two sheets, Ascending and Descending which corresponds to satellite direction. Each sheet contains the time series data for various grids point. Missing data is assigned a number of 999.9. In this section we will read data of one station for all the time, modify the data which is missing, and write in another xls file. We will be using xlsrd library to read data from xls file, and xlwt to write the data to xls file. The xlrd does not read xlsx data file, you should convert the xlsx type of file into xls before reading.

In [ ]:
from google.colab import drive
import xlrd
import numpy as np

drive.mount("/content/gdrive")

fname = "/content/gdrive/My Drive/Colab Notebooks/LearnPython/data/data.xls"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


We create book object by passing the name of xls file to xlrd.open_workbook. The sheet from which we need to read the data is specified using the sheet_by_name.

In [ ]:
book = xlrd.open_workbook(fname)
sheet = book.sheet_by_name('Ascending')

The number of columns and rows in sheets can be checked by using the nrows and ncols attributes respectively.

In [ ]:
print(sheet.nrows)
print(sheet.ncols)

1100
39


Our sheet’s first two rows are heading of table and latitude and longitude, and hence the length of time series data is two lesser than the number of rows. First we create an empty array to store the data, and then we read the data cell by cell using the cell_value. We will be reading the data of grid having latitude equal to 12.4958 and longitude equal to 75.7484, which is in fourth column (indices start with zero).

In [ ]:
sm = np.empty(sheet.nrows-2)
year = np.empty(sheet.nrows-2, int)
month = np.empty(sheet.nrows-2, int)
day = np.empty(sheet.nrows-2, int)
for i in range(sm.shape[0]):
  sm[i] = sheet.cell_value(i+2,27)
  year[i] = sheet.cell_value(i+2,0)
  month[i] = sheet.cell_value(i+2,1)
  day[i] = sheet.cell_value(i+2,2)

# check the output
print(sm)

[ 12.6 999.9  13.2 ...  12.1  12.7  12.8]


We can define "no data" as "NaN"

In [ ]:
sm[sm==999.9] = np.nan
print(sm)

[12.6  nan 13.2 ... 12.1 12.7 12.8]


Now the soil moisture data has nan instead of 999.9 to denote missing values. We will write this soil moisture data into xls file using xlwt library. First we open a workbook, then we add a sheet by name using add_sheet. After this we start writing entries cell by cell. Finally, we save the worksheet using book.save.

In [ ]:
import xlwt

fname = "/content/gdrive/My Drive/Colab Notebooks/LearnPython/data/data1.xls"

book = xlwt.Workbook()
sheet = book.add_sheet('Ascending')
sheet.write(0,0, 'Year')
sheet.write(0,1, 'Month')
sheet.write(0,2, 'Day')
sheet.write(0,3, 'Latitude')
sheet.write(1,3, 'Longitude')

for i in range(len(sm)):
  sheet.write(i+2, 4, sm[i])
  sheet.write(i+2, 0, int(year[i]))
  sheet.write(i+2, 1, int(month[i]))
  sheet.write(i+2, 2, int(day[i]))
  book.save(fname)

You can open the output xls file in Excel (or similar tool) and check the written output.

I have written a library ambhas.xls which provides relatively easy way to read and write the xls data. Library is reproduced below for easy use.

In [ ]:
import numpy as np
import os

class xlsread():
    """
    A class to read data from xls file
    based on the 'xlrd'
    
    Example:
            fname = '/home/tomer/rain_projection/raw_data/a2_0.5.xls'
            foo = xlsread(fname)
            var = foo.get_cells('a3:a5', 'Sheet1')
    """
    
    def __init__(self, fname):
        self.fname = fname
        book = xlrd.open_workbook(self.fname)
        self.sheet_names = book.sheet_names()
        self.book = book
                
    def get_cells(self, cell_range, sheet, dtype='nparray'):
        """
        cell_range: a single cell i.e. 'a2'
                    range of cells i.e. 'a2:f5'
        sheet:  name of the sheet, must be string
        dtype: datatype
               nparray --> numpy array (default)
               list --> a list 
        """
        book = self.book
        sheet = book.sheet_by_name(sheet)
        
        if ':' not in cell_range:
            foo1 = cell_range
            row,col = self.__cell2ind__(foo1)
            data = sheet.cell_value(row,col)
        else:
            foo1, foo2 = cell_range.split(':')
            row1,col1 = self.__cell2ind__(foo1)
            row2,col2 = self.__cell2ind__(foo2)
            
            if row2<row1:
                raise Exception('row_start should be <= row_end')
            if col2<col1:
                raise Exception('col_start should be <= col_end')
            
            data = []
            for i in range(row1,row2+1):
                data_row = []
                for j in range(col1,col2+1):
                    if sheet.cell_value(i,j): # test if the cell is empty
                        data_row.append(sheet.cell_value(i,j))
                    else: 
                        if sheet.cell_value(i,j) == 0:
                            # if the cell is zero fill with zeros
                            data_row.append(sheet.cell_value(i,j))
                        else:
                            # if cell is empty fill with nan
                            data_row.append(np.nan)
                    
                data.append(data_row)
        
        if dtype == 'nparray':
            data = np.array(data)
        elif dtype == 'list':
            pass
        else:
            raise Exception('value error in dtype')
            
        return data

    def __cell2ind__(self,foo):
        """
        given the cell number i.e. (AA100)
        returns the row and column of cell in indices format i.e. 0, 10
        """
        
        # if the column is less than Z
        foo_str = foo[0].lower()
        col = ord(foo_str)-ord('a')
        
        # if the column is more than Z, i.e. AA
        try:
            row = int(foo[1:])-1
        except:
            row = int(foo[2:])-1
            foo_str = foo[1].lower()
            col = (col+1)*26+ ord(foo_str)-ord('a')
        
        return row, col

class xlswrite():
    """
    This saves the array in xls format

    Example:
    var = np.array([[5,10,12],[2,5,6]])
    xls_out_file = xlswrite(var, 'f10', 'Sheet1')
    fname = '/home/tomer/data.xls'
    xls_out_file.save(fname)
    """
    
    def __init__(self, data, cell_start, sheet):
        self.data = data
        self.cell_start = cell_start
        
        # initialize the xlwt     
        book = xlwt.Workbook()
        sheet = book.add_sheet(sheet)
        
        # convert into row and col        
        row, col = self.__cell2ind__(cell_start)
        
        if isinstance(data, str)  or isinstance(data, float) or isinstance(data,int):
            sheet.write(row,col,data)
                
        if data.ndim == 1:
            for i in range(data.shape[0]):
                sheet.write(row+i,col, data[i])
                
        else:
            for i in range(data.shape[0]):
                for j in range(data.shape[1]):
                    sheet.write(row+i, col+j, data[i,j])
        
        self.book = book
    
    def save(self, fname):
        self.book.save(fname)
    
    def __cell2ind__(self, foo):
        """
        given the cell number i.e. (AA100)
        returns the row and column of cell in indices format i.e. 0,10
        """
        
        # if the column is less than Z
        foo_str = foo[0].lower()
        col = ord(foo_str)-ord('a')
        
        # if the column is more than Z, i.e. AA
        try:
            row = int(foo[1:])-1
        except:
            row = int(foo[2:])-1
            foo_str = foo[1].lower()
            col = (col+1)*26+ ord(foo_str)-ord('a')
        
        return row,col

class xlswrite2(xlswrite):
    """
    This saves the array in xls format

    Example:
    var = np.array([[5,10,12],[2,5,6]])
    fname = '/home/tomer/data.xls'    
    xls_out_file = xlswrite2(fname)
    xls_out_file.write(var, 'f10', 'Sheet1')
    xls_out_file.save()
    """
    
    def __init__(self, fname):
        self.fname = fname
                
        # initialize the xlwt     
        self.book = xlwt.Workbook()
        self.sheet_names = []
    
    def write(self, data, cell_start, sheet, dates=False, orientation='row'):
        """
        Input:
            orientation: {row,col}
        """
        book = self.book

        if dates:
            xf = xlwt.easyxf(num_format_str='DD/MM/YYYY')

        if sheet not in self.sheet_names:
            worksheet = book.add_sheet(sheet)
            self.sheet_names.append(sheet)
        else:
            worksheet = book.get_sheet(self.sheet_names.index(sheet))


        # convert into row and col        
        row, col = self.__cell2ind__(cell_start)

        if isinstance(data, str)  or isinstance(data, float) or isinstance(data,int):
            self._write_cell(worksheet, row, col, data, dates)

        elif isinstance(data, np.ndarray):
            if data.ndim == 1:
                for i in range(data.shape[0]):
                    self._write_cell(worksheet, row+i, col, data[i], dates)
            else:
                for i in range(data.shape[0]):
                    for j in range(data.shape[1]):
                        self._write_cell(worksheet, row+i, col+j, data[i,j], dates)

        elif isinstance(data, list):
            for i in range(len(data)):
                if orientation == 'row':
                    self._write_cell(worksheet, row+i, col, data[i], dates)
                elif orientation == 'col':
                    self._write_cell(worksheet, row, col+i, data[i], dates)
                else:
                    print("Orientation has to be either 'row' or 'col' ")

    def _write_cell(self, worksheet, row_cell, col_cell, data_cell, dates):
        if dates:
            xf = xlwt.easyxf(num_format_str='DD/MM/YYYY')
            worksheet.write(row_cell, col_cell, data_cell, xf)
        else:
            worksheet.write(row_cell, col_cell, data_cell)

    def save(self):
        self.book.save(self.fname)

Let us use *xlsread* Class defined above to read the data from *xls* file in the following way.

In [ ]:
fname = "/content/gdrive/My Drive/Colab Notebooks/LearnPython/data/data.xls"
foo = xlsread(fname)
data = foo.get_cells('a3:a5', 'Ascending')
print(data)

[[2007.]
 [2008.]
 [2008.]]


The data to xls file is written in the following way. The data which is written should be a numpy array.

In [ ]:
fname = "/content/gdrive/My Drive/Colab Notebooks/LearnPython/data/data2.xls"

foo = xlswrite(data, 'a3', 'Ascending')
foo.save(fname)

As this library depends upon the xlrd, it also does not read xlsx data file, and you should convert the xlsx type of file into xls before reading.